In [ ]:
import numpy as np
import graphinglib as gl
from astropy.io import fits
from matplotlib.colors import ListedColormap

from src.hdu.map import Map
from src.hdu.grouped_maps import GroupedMaps
from src.coordinates.celestial_coords import RA, DEC

### Fig 1

In [ ]:
data = fits.open(
    "data/output_NGC4696_G235H_F170LP_full_model/NGC4696_G235H_F170LP_full_model_parameter_maps.fits"
)[67].data  # H_2(1-0)
hm = gl.Heatmap(data, origin_position="lower", show_color_bar=False, color_map_range=(-18, -16))
zoomed_hm = gl.Heatmap(data[24:33,24:33], origin_position="lower", show_color_bar=False, color_map_range=(-18, -16))
zoomed_hm_2 = zoomed_hm.copy()
zoomed_hm_2.image = zoomed_hm.image[2:7,2:7]
zoomed_hm_2.color_map = "plasma"

fig = gl.SmartFigure(
    2,
    2,
    remove_x_ticks=True,
    remove_y_ticks=True,
    size=(11, 6),
    width_ratios=[5, 2],
    reference_labels=False,
    width_padding=0,
    height_padding=0,
).set_visual_params(use_latex=True)
fig[:, 0] = [
    hm,
    gl.Text(1, 54, "NGC 4696 ($z=0.0104$)", h_align="left"),
    gl.Text(1, 52, "Central dominant galaxy in the Centaurus Cluster", h_align="left"),
    gl.Text(1, 3, r"H$\alpha$ HST ...", h_align="left"),
    gl.Text(1, 1, r"Chandra X-ray (blue) ...", h_align="left"),
    gl.Text(28, 35, r"[Chandra X-ray + H$\alpha$ HST image"+"\nof the whole cluster]", font_size=20),
    gl.Line((45, 1), (55, 1), capped_line=True, cap_width=0.5, width=1),
    gl.Text(50, 2, "x kpc"),
    gl.Rectangle(23.5, 23.5, 9, 9, fill=False),
    gl.Polygon([[9+23.5, 23.5], [9+23.5, 9+23.5], [60, 60], [60, 28.7]], fill=False),
]
fig[0, 1] = [
    zoomed_hm,
    gl.Text(4, 3, "[Identify with\narrows the AGN\nand filaments]", font_size=15),
    gl.Text(6.5, 7.3, "x kpc"),
    gl.Line((5, 7), (8, 7), capped_line=True, cap_width=0.5, width=1),
    gl.Rectangle(1.5, 1.5, 5, 5, fill=False),
    gl.Polygon([[-0.5, -0.5], [1.5, 1.5], [6.5, 1.5], [9.5, -1.5]], fill=False),
]
fig[1, 1] = [
    zoomed_hm_2,
    gl.Rectangle(0.5, 0.5, 3, 3, fill=False),
    gl.Text(2, 4, "[MUSE velocity field]", font_size=15),
    gl.Text(2, 3.1, "[NIRSpec's FOV]", font_size=15),
    gl.Line((2, 0), (4, 0), capped_line=True, cap_width=0.5, width=1),
    gl.Text(3, 0.2, "x kpc"),
]
fig.show().save("figures/article/fig_1.pdf", dpi=600)

### Fig 2

In [ ]:
gm = GroupedMaps.load_from_loki("data/output_NGC4696_G235H_F170LP_full_model/NGC4696_G235H_F170LP_full_model_parameter_maps.fits")
header = gm["LINES.H210_Q7.TOTAL_FLUX"].header
world_coords = [RA.from_sexagesimal("12:48:49.2609").degrees, DEC.from_sexagesimal("-41:18:39.417").degrees]
coords = header.world_to_pixel(world_coords)[0][::-1]  # x, y
# print(coords)
# Rotate this coord
# theta = np.radians(-47)
# rotated_coords = np.array([[np.cos(theta), -np.sin(theta)], [np.sin(theta), np.cos(theta)]]) @ coords
# print(rotated_coords)

lines = [f"LINES.H210_Q7.{p}" for p in ["TOTAL_FLUX", "VPEAK", "1.FWHM"]]
lines += [f"CONTINUUM.STELLAR_{p}" for p in ["POPULATIONS.MASS", "KINEMATICS.VEL", "KINEMATICS.VDISP"]]
# H210_Q7 [327, 336, 89] and stars [429, 5, 6]
hms = []
for i, line in enumerate(lines):
    map_ = gm[line]
    # map_.data[*np.round(coords).astype(int)] = np.nan

    if i in [0, 3]:
        if i == 0: cbar_label = r"$\log_{10}(F/$ erg s$^{-1}$ cm$^{-2})$"
        else: cbar_label = r"$\log_{10}(M/M_{\odot})$"
        cmap = "plasma"
    elif i in [1, 4]:
        cbar_label = "velocity $v$ (km s$^{-1}$)"
        cmap = "coolwarm"
    else:
        if i == 2:
            map_ /= 2 * np.sqrt(2 * np.log(2))  # FWHM to sigma
        cbar_label = r"velocity dispersion $\sigma$ (km s$^{-1}$)"
        cmap = "viridis"

    hm = map_.rotate_field()
    hm.color_map = cmap
    if i < 3: hm.set_color_bar_params(position="top", label=cbar_label)
    else: hm.set_color_bar_params(position="bottom", label=cbar_label)
    hms.append(hm)

arrows = [
    gl.Arrow((7.8+20, 2-23), (12.8+20, 7-23), "black"),
    gl.Arrow((8.2+20, 2-23), (3.2+20, 7-23), "black"),
    gl.Text(13+20, 7-23, "N", color="black", font_size=15, h_align="center", v_align="bottom"),
    gl.Text(3+20, 7-23, "E", color="black", font_size=15, h_align="center", v_align="bottom"),
]
AGN_pos = gl.Point(40.3, 0, marker_style="x", color="red", marker_size=50)
AGN_text = gl.Text(37.3, 2, r"$\textbf{AGN}$", color="red", font_size=14, h_align="center", v_align="center")
radio_contours = gl.Contour.from_function(
    # lambda x, y: ((x-40.3)**2 + (y-0)**2)**0.5,
    lambda x, y: np.exp(-0.5*(((x-40.3)/3)**2 + ((y-0)/1.5)**2)),
    (20, 60),
    (-23, 20),
    2,
    ListedColormap(["red"]),
    show_color_bar=False,
    filled=False,
    alpha=0.5
)
fig = gl.SmartFigure(
    2,
    3,
    # aspect_ratio=1,
    size=(9.12, 8),
    remove_x_ticks=True,
    remove_y_ticks=True,
    elements=hms,
    # reference_labels_loc=(0.2, -0.5),
    reference_labels_loc=(0.03, -0.16),
    width_padding=0,
    height_padding=0,
    x_lim=(20, 60),
    y_lim=(-23, 20),
)
fig.add_elements(*[[*arrows, AGN_pos, AGN_text, radio_contours]]*6)#.set_reference_labels_params(color="black", font_size=15)
hms[0].color_map_range = -18.27, -17.53
hms[1].color_map_range = -460, 250
hms[2].color_map_range = 30, 340
hms[3].color_map_range = 6.5, 7.24
hms[4].color_map_range = -50, 50
hms[5].color_map_range = 230, 325
%matplotlib inline
fig.set_visual_params(use_latex=True).show()#.save("figures/article/fig_2_v3.pdf", dpi=600)

In [ ]:
left_fig = gl.SmartFigure(
    2,
    1,
    aspect_ratio=1,
    remove_x_ticks=True,
    remove_y_ticks=True,
    elements=[flux_hm, flux_hm],
    reference_label_loc="inside",
).add_elements(*[arrows]*2)
middle_right_fig = gl.SmartFigure(
    2,
    2,
    aspect_ratio=1,
    remove_x_ticks=True,
    remove_y_ticks=True,
    elements=[vel_hm_0, sig_hm_0, vel_hm, sig_hm],
    reference_label_loc="inside",
    height_ratios=[5, 6.5]
).add_elements(*[arrows]*4)
global_fig = gl.SmartFigure(
    1,
    2,
    size=(10, 6),
    elements=[left_fig, middle_right_fig],
    width_ratios=[1, 2],
).set_visual_params(use_latex=True).show()#.save("figures/article/fig_2.pdf", dpi=600)

### Tests

In [ ]:
from src.tools.miscellaneous import get_pdf_image_as_array

# lines = [f"H210_{i}" for i in ["O2", "O3", "O4", "Q6", "Q7", "Q8", "Q9", "S0", "S1", "S2", "S3", "S4", "S5", "S6", "S7", "S8"]]
lines = [f"H210_{i}" for i in ["O2", "Q7", "S4", "S5"]]
hms = []
subtitles = []
path = "data/output_NGC4696_G235H_F170LP_full_model/param_maps/lines"
for line in lines:
    for suffix in ["1.flux", "1.voff", "1.fwhm"]:
        hms.append(gl.Heatmap(
            get_pdf_image_as_array(f"{path}/{line}/{line}.{suffix}.pdf"),
            show_color_bar=False,
        ))
        subtitles.append(f"{line} {suffix.lstrip("1.")}")

num_rows = len(hms) // 3
fig = gl.SmartFigure(
    num_rows,
    3,
    elements=hms,
    subtitles=subtitles,
    size=(10, 2.75*num_rows),
    remove_axes=True,
    reference_labels=False,
    aspect_ratio=1,
).save("figures/tests/relevant_H210.pdf")
# fig = gl.SmartFigure(
#     3,
#     num_rows,
#     elements=np.array(hms, dtype=object).reshape((4, 3)).T.flatten(),
#     subtitles=np.array(subtitles, dtype=object).reshape((4, 3)).T.flatten(),
#     # size=(10, 2.75*num_rows),
#     size=(2.75*num_rows, 7),
#     remove_axes=True,
#     reference_labels=False,
#     aspect_ratio=1,
# ).save("t.pdf")